<a href="https://colab.research.google.com/github/sugampath/BERT-result/blob/master/Bertclean%20data%2080%20%2020%20midterm%204%20epoch%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 2.1MB 20.6MB/s 
     |████████████████████████████████| 3.3MB 43.0MB/s 
     |████████████████████████████████| 901kB 51.1MB/s 


In [3]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler,TensorDataset
from transformers import AdamW, get_linear_schedule_with_warmup
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import seaborn as sns

Read the data




In [4]:
data=pd.read_csv("/content/drive/MyDrive/Thesis/dataset.csv") 

In [5]:
data.rename(columns = {"label":"category"}, inplace = True)

In [6]:
data.head(10)

,feature,category
0,सुदूरपश्चिम समाज कतारको दोस्रो साधारणसभा सम्पन...,diaspora
1,अमेरिकाबाट गायिका थापाको देशभक्ति गीत (भिडियो ...,diaspora
2,आदिवासी जनजाति मञ्च अष्ट्रेलियामा नयाँ कार्यसम...,diaspora
3,नेपाल फेस्टिवल मेलवर्नः नेपालका पर्यटनमन्त्री ...,diaspora
4,नेपाल फेष्टिबलको धन्यवाद ज्ञापन सम्पन्न\n\n१२ ...,diaspora
5,आगलागी पीडित थापालाई भेटे एनआरएन अध्यक्षले\n\n...,diaspora
6,फिजीका नेपालीलाई नेपाली भाषा पढाइने\nभीमसेन सा...,diaspora
7,अष्ट्रेलियामा नेपालीको पहिलो संस्थाले पायो नया...,diaspora
8,कराते खेलाडी ठकुरी ‘अमेरिका गट ट्यालेन्ट’मा जा...,diaspora
9,ईयूका लागि नेपाली राजदूत नियुक्त भएपछि थापाले ...,diaspora


In [7]:
data["feature"]

0        सुदूरपश्चिम समाज कतारको दोस्रो साधारणसभा सम्पन...
1        अमेरिकाबाट गायिका थापाको देशभक्ति गीत (भिडियो ...
2        आदिवासी जनजाति मञ्च अष्ट्रेलियामा नयाँ कार्यसम...
3        नेपाल फेस्टिवल मेलवर्नः नेपालका पर्यटनमन्त्री ...
4        नेपाल फेष्टिबलको धन्यवाद ज्ञापन सम्पन्न\n\n१२ ...
                               ...                        
19985    बुटवलमा सातौं क्यान इन्फोटेक : १ लाख दर्शक र २...
19986    बदलिँदैछ फेसबुकको अनुहारः तन्नेरी छोड्दै, पाका...
19987    आयो स्मार्ट चश्मा, जसले अपराधी पत्ता लगाउनेछ\n...
19988    नोकिया ८ मा एन्ड्रोइड ओरियो अपडेट\nनोकियाको फ्...
19989    गुगल क्रोममा नयाँ फिल्टर, अनलाइन विज्ञापन ब्लक...
Name: feature, Length: 19990, dtype: object

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
stop_words = stopwords.words("nepali")
data["feature"] = data["feature"].apply(lambda x: re.sub('[।(),०-९<<?!,—,–,/,,’,,,“,‘,:,\u200d,”,\n,\xa0,","]', " ", x))
data["feature"] = data["feature"].apply(lambda x: " ".join([i for i in x.split()if i not in (stop_words)]))

In [10]:
data["feature"][200]

'साउदी अरबको जेद्दामा मनाइयो ल्होछार जेद्दा मौलिक धर्म संस्कृति परम्परालाई जगेर्ना निरन्तर उत्थान सम्बर्द्धन एवं प्रबर्द्धन साउदी अरबको जेद्धा आसपासमा तामाङ श्रमिकहरूले बृहत रूपमा सोनाम ल्होछार मनाएका ज्या ल्हो चरा बर्गको वर्ष आगमनलाई हर्षोउल्लासका मनाइयो नेपाल तामाङ घेदुङ प्रवास शाखा साउदी अरबका अध्यक्ष विगेन्द्रसिंह वाइवाको सभापतित्वमा सम्पन्न उक्त कार्यक्रमलाई घेदुङका महासचिव नवराज घिसिङले संचालन महावाणीज्य दूतावासका कार्यवाहक महावाणीज्यदूत रेवती रमण पौडेलको प्रमुख आतिथ्यता कार्यक्रममा अतिथिहरू वाणीज्यदूत मनहरी दंगाल श्रम सहचारी भोजराज घिमिरे आदिबासी जनजाति महासंघ साउदी अरबका केन्द्रिय उपाध्यक्ष बालकुमार राई आदिबासी जनजाति महासंघ क्षेत्रीय समन्वय समिति जेद्धाका अध्यक्ष ङिमा साङगे शेर्पा लगायतको उपस्थिति कार्यक्रममा गैरआवासीय नेपाली संघ साउदी अरबका उपाध्यक्ष ध्रुब रिजाल तमु धी जेद्धाका अध्यक्ष वुद्धीलाल गुरूङ नेपाली युवा क्लब जेद्धाका अध्यक्ष कृष्ण भट्टराई आवासीय नेपाली संघ क्षेत्रीय समिति जेद्धाका अध्यक्ष भिम गुरूङ समाजसेवी कलाकार धनबहादुर गुरुङ बरिष्ठ समाजसेवी बेद कुमार गुरुङ समाजस

In [11]:
data.shape

(19990, 2)

In [12]:
data["category"].value_counts()

health           2000
society          2000
diaspora         2000
technology       2000
sports           2000
entertainment    2000
opinion          2000
economy          2000
politics         2000
international    1990
Name: category, dtype: int64

In [13]:
possible_labels = data.category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'diaspora': 0,
 'economy': 1,
 'entertainment': 2,
 'health': 3,
 'international': 4,
 'opinion': 5,
 'politics': 6,
 'society': 7,
 'sports': 8,
 'technology': 9}

In [14]:
possible_labels

array(['diaspora', 'economy', 'entertainment', 'health', 'international',
       'opinion', 'politics', 'society', 'sports', 'technology'],
      dtype=object)

In [15]:
data["label"]=data.category.replace(label_dict)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_val, y_train, y_val = train_test_split(data.index.values, 
                                                  data.label.values, 
                                                  test_size=0.2, 
                                                  random_state=42, 
                                                  stratify=data.label.values)

In [18]:
X_train,X_val
print(X_train.shape)
print(X_val.shape)

(15992,)
(3998,)


In [19]:
data["data_type"]=["not_set"]*data.shape[0]
data.loc[X_train,"data_type"]="train"
data.loc[X_val,"data_type"]="Val"

In [20]:
data.groupby(["category","label","data_type"]).count()

feature
category      label data_type         
diaspora      0     Val            400
                    train         1600
economy       1     Val            400
                    train         1600
entertainment 2     Val            400
                    train         1600
health        3     Val            400
                    train         1600
international 4     Val            398
                    train         1592
opinion       5     Val            400
                    train         1600
politics      6     Val            400
                    train         1600
society       7     Val            400
                    train         1600
sports        8     Val            400
                    train         1600
technology    9     Val            400
                    train         1600

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',truncation= True)

In [22]:
encoded_data_train = tokenizer.batch_encode_plus(
    data[data.data_type=='train'].feature.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt')


encoded_data_val = tokenizer.batch_encode_plus(
    data[data.data_type=='Val'].feature.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt')

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(data[data.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(data[data.data_type=='Val'].label.values)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [23]:
input_ids_train

tensor([[   101,    898,  14070,  ...,      0,      0,      0],
        [   101,  18263, 102116,  ...,      0,      0,      0],
        [   101,  29953,    886,  ...,      0,      0,      0],
        ...,
        [   101,    898,  60439,  ...,      0,      0,      0],
        [   101,    884,  69897,  ...,  44989,  12512,    102],
        [   101,    867,  78660,  ...,      0,      0,      0]])

In [24]:

input_ids_train[200]

tensor([   101,  64821,  27117, 102116,  10914,  13088,  51283,  40153,  16940,
           851,  55874,  89070,  77073,  37038,  21835,  28106,  87517,  64821,
         27117,    852,  43263,  40340,  40265,    889,  17110,  25936,  11208,
         12512,    880,  19885,  12878,  27841,  12347,  21147,  61809,  12512,
         29953,  50363,    885,  27373,  25561,  63911,  13088,  51283,  37038,
         84456,  74157,    898,  11208,  79371,  13432, 104741,  15399,  64821,
         27117,  21582,  12347,  26876,    898,  13841,  14070, 102299,  63911,
         13088,  51283,  15399, 110126, 101034,  41510,  13665,    851,  55874,
         89070,  77073,    886,  11554,  18351,  18321, 111192,  20161,    880,
         19885,  12878,  27841,  12347,  64821,  27117,  21582,  12347,  13088,
         51283,  15399, 110126, 101034,  41510,  48558,  49050,  97571,  13718,
           885,  17110,  11714,  12347,    851,  55874,  89070,  77073,    898,
         45753,  68053,  22949,  37245, 

In [25]:
input_ids_train.shape

torch.Size([15992, 512])

In [26]:
attention_masks_train

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])

In [27]:
attention_masks_train.shape

torch.Size([15992, 512])

In [28]:
attention_masks_train[2]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [29]:
labels_train

tensor([0, 0, 0,  ..., 9, 9, 9])

In [30]:
labels_train.shape

torch.Size([15992])

In [31]:
labels_train[43]

tensor(0)

INPUT AND ATTENTION FORMAT


In [32]:
encoded_data_train

{'input_ids': tensor([[   101,    898,  14070,  ...,      0,      0,      0],
        [   101,  18263, 102116,  ...,      0,      0,      0],
        [   101,  29953,    886,  ...,      0,      0,      0],
        ...,
        [   101,    898,  60439,  ...,      0,      0,      0],
        [   101,    884,  69897,  ...,  44989,  12512,    102],
        [   101,    867,  78660,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [33]:
type(encoded_data_train)

transformers.tokenization_utils_base.BatchEncoding

In [34]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [35]:
len(dataset_train),len(dataset_val)

(15992, 3998)

In [36]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [73]:
batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [74]:
 sampler=SequentialSampler(dataset_val)
 sampler

In [75]:

optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  eps=1e-8)

In [76]:
epochs = 4

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [77]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [78]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [81]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [82]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    FILE="model.pth"   
    torch.save(model.state_dict(),FILE)
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 0.8239052807817238
Validation loss: 0.7032982454565249
F1 Score (Weighted): 0.8612988997006142



Epoch 2
Training loss: 0.5104615849455522
Validation loss: 0.5625907885780128
F1 Score (Weighted): 0.8877263831009911



Epoch 3
Training loss: 0.3050442787330075
Validation loss: 0.5278555909508207
F1 Score (Weighted): 0.9108289242512027



Epoch 4
Training loss: 0.17899499189523052
Validation loss: 0.5011399301699514
F1 Score (Weighted): 0.9203744405865029



In [83]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [84]:
model.load_state_dict(torch.load('/content/model.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [85]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [86]:
true_vals[1:50]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [87]:
accuracy_per_class(predictions, true_vals)


Class: diaspora
Accuracy: 368/400

Class: economy
Accuracy: 360/400

Class: entertainment
Accuracy: 372/400

Class: health
Accuracy: 362/400

Class: international
Accuracy: 373/398

Class: opinion
Accuracy: 365/400

Class: politics
Accuracy: 378/400

Class: society
Accuracy: 324/400

Class: sports
Accuracy: 390/400

Class: technology
Accuracy: 388/400



In [88]:
predlabel=np.argmax(predictions,axis=1)

In [89]:
predlabel


array([0, 0, 0, ..., 9, 9, 9])

In [90]:
len(predlabel)

3998

In [91]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
print(classification_report(true_vals,predlabel,digits =2,target_names=possible_labels))


               precision    recall  f1-score   support

     diaspora       0.92      0.92      0.92       400
      economy       0.84      0.90      0.87       400
entertainment       0.91      0.93      0.92       400
       health       0.92      0.91      0.91       400
international       0.94      0.94      0.94       398
      opinion       0.94      0.91      0.93       400
     politics       0.95      0.94      0.95       400
      society       0.86      0.81      0.84       400
       sports       0.98      0.97      0.98       400
   technology       0.95      0.97      0.96       400

     accuracy                           0.92      3998
    macro avg       0.92      0.92      0.92      3998
 weighted avg       0.92      0.92      0.92      3998



In [92]:
print(confusion_matrix(true_vals,predlabel))

[[368  10   6   1   0   7   1   6   1   0]
 [  4 360   4   4   0   0   0  21   0   7]
 [  9   3 372   1   1   4   4   3   2   1]
 [  0   8   4 362   7   4   1  11   0   3]
 [  2   1   4   3 373   3   0   1   2   9]
 [  3   5  10   2   6 365   6   3   0   0]
 [  1   1   5   3   2   2 378   6   2   0]
 [ 11  34   4  15   1   3   7 324   1   0]
 [  3   0   2   0   5   0   0   0 390   0]
 [  1   6   0   1   3   0   0   1   0 388]]
